# Building Agents That Use Code

This notebook is part of the [Hugging Face Agents Course](https://www.hf.co/learn/agents-course), a free Course from beginner to expert, where you learn to build Agents.

![Agents course share](https://huggingface.co/datasets/agents-course/course-images/resolve/main/en/communication/share.png)

## Let's install the dependencies and login to our HF account to access the Inference API

If you haven't installed `smolagents` yet, you can do so by running the following command:

In [ ]:
# !pip install smolagents

Let's also login to the Hugging Face Hub to have access to the Inference API.

In [2]:
# from huggingface_hub import notebook_login

# notebook_login()

# Model setup instead

from smolagents import LiteLLMModel

OLLAMA_MODEL_7B_Q4_K_M = "qwen2.5-coder:7b-instruct-q4_K_M"

OLLAMA_MODEL_14B_Q4_K_M = "qwen2.5-coder:14b-instruct-q4_K_M"

OLLAMA_MODEL_32B_Q4_K_M = "qwen2.5-coder:32b-instruct-q4_K_M"

MODEL_ID = "/".join(["ollama_chat", OLLAMA_MODEL_7B_Q4_K_M])

lite_model = LiteLLMModel(
    model_id=MODEL_ID,
    api_base="http://localhost:11434",
)


## The `@tool` Decorator  

### Generating a tool that retrieves the highest-rated catering

Let's imagine that Alfred has already decided on the menu for the party, but now he needs help preparing food for such a large number of guests. To do so, he would like to hire a catering service and needs to identify the highest-rated options available. Alfred can leverage a tool to search for the best catering services in his area.

Below is an example of how Alfred can use the `@tool` decorator to make this happen:

In [2]:
from smolagents import CodeAgent, LiteLLMModel, tool

# Let's pretend we have a function that fetches the highest-rated catering services.
@tool
def catering_service_tool(query: str) -> str:
    """
    This tool returns the highest-rated catering service in Gotham City.

    Args:
        query: A search term for finding catering services.
    """
    # Example list of catering services and their ratings
    services = {
        "Gotham Catering Co.": 4.9,
        "Wayne Manor Catering": 4.8,
        "Gotham City Events": 4.7,
    }

    # Find the highest rated catering service (simulating search query filtering)
    best_service = max(services, key=services.get)

    return best_service

if not lite_model:
    raise ValueError("Run cell near top of notebook to ensure lite_model instance")

agent = CodeAgent(tools=[catering_service_tool], model=lite_model)

# Run the agent to find the best catering service
result = agent.run(
    "Can you give me the name of the highest-rated catering service in Gotham City?"
)

print(result)

╭──────────────────────────────────────────────────── New run ────────────────────────────────────────────────────╮
│                                                                                                                 │
│ Can you give me the name of the highest-rated catering service in Gotham City?                                  │
│                                                                                                                 │
╰─ LiteLLMModel - ollama_chat/qwen2.5-coder:7b-instruct-q4_K_M ───────────────────────────────────────────────────╯

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 1 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

─ Executing parsed code: ──────────────────────────────────────────────────────────────────────────────────────── 
  top_catering_service = catering_service_tool(query="best catering service in Gotham City")                       
  final_answer(top_catering_service)                                                                               
 ─────────────────────────────────────────────────────────────────────────────────────────────────────────────────

Out - Final answer: Gotham Catering Co.

[Step 1: Duration 203.28 seconds| Input tokens: 2,043 | Output tokens: 75]

Gotham Catering Co.


## Defining a Tool as a Python Class  

### Generating a tool to generate ideas about the superhero-themed party

Alfred's party at the mansion is a **superhero-themed event**, but he needs some creative ideas to make it truly special. As a fantastic host, he wants to surprise the guests with a unique theme.

To do this, he can use an agent that generates superhero-themed party ideas based on a given category. This way, Alfred can find the perfect party theme to wow his guests.

In [4]:
from smolagents import Tool, CodeAgent, LiteLLMModel

class SuperheroPartyThemeTool(Tool):
    name = "superhero_party_theme_generator"
    description = """
    This tool suggests creative superhero-themed party ideas based on a category.
    It returns a unique party theme idea."""

    inputs = {
        "category": {
            "type": "string",
            "description": "The type of superhero party (e.g., 'classic heroes', 'villain masquerade', 'futuristic Gotham').",
        }
    }

    output_type = "string"

    def forward(self, category: str):
        themes = {
            "classic heroes": "Justice League Gala: Guests come dressed as their favorite DC heroes with themed cocktails like 'The Kryptonite Punch'.",
            "villain masquerade": "Gotham Rogues' Ball: A mysterious masquerade where guests dress as classic Batman villains.",
            "futuristic Gotham": "Neo-Gotham Night: A cyberpunk-style party inspired by Batman Beyond, with neon decorations and futuristic gadgets."
        }

        return themes.get(category.lower(), "Themed party idea not found. Try 'classic heroes', 'villain masquerade', or 'futuristic Gotham'.")

# Instantiate the tool
party_theme_tool = SuperheroPartyThemeTool()

if not lite_model:
    raise ValueError("Run cell near top of notebook to ensure lite_model instance")

agent = CodeAgent(tools=[party_theme_tool], model=lite_model)

# Run the agent to generate a party theme idea
result = agent.run(
    "What would be a good superhero party idea for a 'villain masquerade' theme?"
)

print(result)

╭──────────────────────────────────────────────────── New run ────────────────────────────────────────────────────╮
│                                                                                                                 │
│ What would be a good superhero party idea for a 'villain masquerade' theme?                                     │
│                                                                                                                 │
╰─ LiteLLMModel - ollama_chat/qwen2.5-coder:7b-instruct-q4_K_M ───────────────────────────────────────────────────╯

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 1 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

Error in code parsing:
Your code snippet is invalid, because the regex pattern ```(?:py|python)?\s*\n(.*?)\n``` was not found in it.
            Here is your code snippet:
            A villain-themed superhero party could be fun and exciting! Here are some ideas to consider:

1. Masked Races: Have a race where participants wear masks of their favorite villains. You can create obstacles 
that require teamwork or individual skill.

2. Costume Contest: Encourage guests to come dressed as their favorite supervillains. You can have judges who award
prizes for the most creative and well-crafted costumes.

3. Villainous Activities: Set up stations where guests can participate in activities related to villainy, such as 
making evil-looking masks, creating a fake crime scene, or practicing bad guy dialogues.

4. Power-Ups: Give each guest a power-up that makes them feel like a super-powered villain. These could be things 
like glowing visors, invisible wands, or super strength elixirs.

5. Secret Mission: Create a secret mission where guests have to work together to stop a supervillain plot. This 
could involve solving puzzles, decoding messages, or outsmarting a group of evil henchmen.

6. Costume Contest: Encourage guests to come dressed as their favorite supervillains. You can have judges who award
prizes for the most creative and well-crafted costumes.

7. Villainous Games: Set up games that require teamwork, strategy, and problem-solving skills. For example, you 
could play a game where guests have to outsmart a group of evil henchmen or solve puzzles to break into a lair.

8. Movie Night: Watch a movie with a villain as the main character. You can pick a movie that's appropriate for all
ages, and then let guests choose their favorite supervillain as the character they want to be during the party.

Overall, the key to a successful villain-themed superhero party is to have fun and get creative! Encourage guests 
to dress up in fun costumes and participate in exciting activities that will make them feel like part of a 
super-powered team.
            Make sure to include code with the correct pattern, for instance:
            Thoughts: Your thoughts
            Code:
            ```py
            # Your python code here
            ```<end_code>
Make sure to provide correct code blobs.

[Step 1: Duration 181.47 seconds| Input tokens: 31 | Output tokens: 392]

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 2 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

...

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 4 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

─ Executing parsed code: ──────────────────────────────────────────────────────────────────────────────────────── 
  final_answer("I don't have enough information to answer your question.")                                         
 ─────────────────────────────────────────────────────────────────────────────────────────────────────────────────

Out - Final answer: I don't have enough information to answer your question.

[Step 4: Duration 196.96 seconds| Input tokens: 6,175 | Output tokens: 776]

I don't have enough information to answer your question.


## Sharing a Tool to the Hub

Sharing your custom tool with the community is easy! Simply upload it to your Hugging Face account using the `push_to_hub()` method.

For instance, Alfred can share his `catering_service_tool` to help others find the best catering services in Gotham. Here's how to do it:

In [6]:
import os
party_theme_tool.push_to_hub("mikejay14/unit_2_1_catering_service_tool", token=os.environ['HF_TOKEN'])

README.md:   0%|          | 0.00/250 [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/spaces/mikejay14/unit_2_1_catering_service_tool/commit/f8f13638a328d4876a6ec1742c605603552d36cf', commit_message='Upload tool', commit_description='', oid='f8f13638a328d4876a6ec1742c605603552d36cf', pr_url=None, repo_url=RepoUrl('https://huggingface.co/spaces/mikejay14/unit_2_1_catering_service_tool', endpoint='https://huggingface.co', repo_type='space', repo_id='mikejay14/unit_2_1_catering_service_tool'), pr_revision=None, pr_num=None)

## Importing a Tool from the Hub

You can easily import tools created by other users using the `load_tool()` function. For example, Alfred might want to generate a promotional image for the party using AI. Instead of building a tool from scratch, he can leverage a predefined one from the community:

In [ ]:
from smolagents import load_tool, CodeAgent, LiteLLMModel

image_generation_tool = load_tool(
    "m-ric/text-to-image",
    trust_remote_code=True
)

if not lite_model:
    raise ValueError("Run cell near top of notebook to ensure lite_model instance")

agent = CodeAgent(
    tools=[image_generation_tool],
    model=lite_model
)

agent.run("Generate an image of a luxurious superhero-themed party at Wayne Manor with made-up superheros.")

## Importing a Hugging Face Space as a Tool

You can also import a HF Space as a tool using `Tool.from_space()`. This opens up possibilities for integrating with thousands of spaces from the community for tasks from image generation to data analysis.

The tool will connect with the spaces Gradio backend using the `gradio_client`, so make sure to install it via `pip` if you don't have it already. For the party, Alfred can also use a HF Space directly for the generation of the previous annoucement AI-generated image. Let's build it!

In [ ]:
#!pip install gradio_client

In [ ]:
from smolagents import CodeAgent, LiteLLMModel, Tool

image_generation_tool = Tool.from_space(
    "black-forest-labs/FLUX.1-schnell",
    name="image_generator",
    description="Generate an image from a prompt"
)

if not lite_model:
    raise ValueError("Run cell near top of notebook to ensure lite_model instance")

agent = CodeAgent(tools=[image_generation_tool], model=lite_model)

agent.run(
    "Improve this prompt, then generate an image of it.",
    additional_args={'user_prompt': 'A grand superhero-themed party at Wayne Manor, with Alfred overseeing a luxurious gala'}
)

In [ ]:
from PIL import Image as PILImage
import matplotlib.pyplot as plt

image_path = '/tmp/gradio/d5a8dfbade97e9b9d99f78d5ccaa73db6d4b8dc428662f2f25bde1de1bd77b81/image.webp'

img = PILImage.open(image_path)
img

## Importing a LangChain Tool

These tools need a [SerpApi API Key](https://serpapi.com/).

You can easily load LangChain tools using the `Tool.from_langchain()` method. Alfred, ever the perfectionist, is preparing for a spectacular superhero night at Wayne Manor while the Waynes are away. To make sure every detail exceeds expectations, he taps into LangChain tools to find top-tier entertainment ideas.

By using `Tool.from_langchain()`, Alfred effortlessly adds advanced search functionalities to his smolagent, enabling him to discover exclusive party ideas and services with just a few commands.

Here's how he does it:

In [ ]:
#!pip install langchain-community google-search-results

In [ ]:
#from google.colab import userdata
#import os
#os.environ["SERPAPI_API_KEY"] = userdata.get('SERPAPI_API_KEY')

In [ ]:
#from langchain.agents import load_tools
from smolagents import CodeAgent, Tool
from langchain_community.utilities import SearxSearchWrapper

lc_searchtool = SearxSearchWrapper(searx_host="http://localhost:8080") # accept k=10 default

#search_tool = Tool.from_langchain(load_tools(["serpapi"])[0])

search_tool = Tool.from_langchain(lc_searchtool)

if not lite_model:
    raise ValueError("Run cell near top of notebook to ensure lite_model instance")

agent = CodeAgent(
    tools=[search_tool],
    model=lite_model)

agent.run("Search for luxury entertainment ideas for a superhero-themed event, such as live performances and interactive experiences.")

With this setup, Alfred can quickly discover luxurious entertainment options, ensuring Gotham's elite guests have an unforgettable experience. This tool helps him curate the perfect superhero-themed event for Wayne Manor! 🎉